-sandbox
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Querying Data Lakes with DataFrames

Apache Spark&trade; and Databricks&reg; make it easy to work with hierarchical data, such as nested JSON records.

Perform exploratory data analysis (EDA) to gain insights from a Data Lake.

## In this lesson you:
* Use DataFrames to query a Data Lake.
* Clean messy data sets.
* Join two cleaned data sets.

## Audience
* Primary Audience: Data Engineers and Data Scientists
* Secondary Audience: Data Analysts

## Prerequisites
* Web browser: Chrome or Firefox
* Lesson: <a href="$./02-Querying-Files">Querying Files with SQL</a>

## Data Lakes

Companies frequently store thousands of large data files gathered from various teams and departments, typically using a diverse variety of formats including CSV, JSON, and XML.  Data scientists often wish to extract insights from this data.

The classic approach to querying this data is to load it into a central database called a <b>data warehouse</b>.  Traditionally, data engineers must design the schema for the central database, extract the data from the various data sources, transform the data to fit the warehouse schema, and load it into the central database.
A data scientist can then query the data warehouse directly or query smaller data sets created to optimize specific types of queries.

The classic data warehouse approach works well but requires a great deal of up front effort to design and populate schemas.  It also limits historical data, which is constrained to only the data that fits the warehouse’s schema.

An alternative approach is a <b>Data Lake</b>, which:

* Is a storage repository that cheaply stores a vast amount of raw data in its native format.
* Consists of current and historical data dumps in various formats including XML, JSON, CSV, Parquet, etc.
* May contain operational relational databases with live transactional data.

Spark is ideal for querying Data Lakes. Use Spark DataFrames to read directly from raw files and then execute queries to join and aggregate the data.

This lesson illustrates that once two DataFrames are created (independent of their underlying file type), we can perform a variety of operations on them including joins, nested queries, and many others.

### Getting Started

Run the following cell to configure our "classroom."

In [5]:
%run "./Includes/Classroom-Setup"

<iframe  
src="//fast.wistia.net/embed/iframe/xtpxgg1cbv?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/xtpxgg1cbv?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

## Looking at our Data Lake

Start by reviewing which files are in our Data Lake.

In `dbfs:/mnt/training/crime-data-2016`, there are Parquet files containing 2016 crime data from several United States cities.

In the cell below we have data for Boston, Chicago, New Orleans, and more.

<iframe  
src="//fast.wistia.net/embed/iframe/l9g7bdhfle?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/l9g7bdhfle?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

In [9]:
%fs ls /mnt/training/crime-data-2016

path,name,size
dbfs:/mnt/training/crime-data-2016/Crime-Data-Boston-2016.parquet/,Crime-Data-Boston-2016.parquet/,0
dbfs:/mnt/training/crime-data-2016/Crime-Data-Chicago-2016.parquet/,Crime-Data-Chicago-2016.parquet/,0
dbfs:/mnt/training/crime-data-2016/Crime-Data-Dallas-2016.parquet/,Crime-Data-Dallas-2016.parquet/,0
dbfs:/mnt/training/crime-data-2016/Crime-Data-Los-Angeles-2016.parquet/,Crime-Data-Los-Angeles-2016.parquet/,0
dbfs:/mnt/training/crime-data-2016/Crime-Data-New-Orleans-2016.parquet/,Crime-Data-New-Orleans-2016.parquet/,0
dbfs:/mnt/training/crime-data-2016/Crime-Data-New-York-2016.parquet/,Crime-Data-New-York-2016.parquet/,0
dbfs:/mnt/training/crime-data-2016/Crime-Data-Philadelphia-2016.parquet/,Crime-Data-Philadelphia-2016.parquet/,0


The next step in looking at the data is to create a DataFrame for each file.  

Start by creating a DataFrame of the data from New York and then Boston:

| City          | Table Name              | Path to DBFS file
| ------------- | ----------------------- | -----------------
| **New York**  | `CrimeDataNewYork`      | `dbfs:/mnt/training/crime-data-2016/Crime-Data-New-York-2016.parquet`
| **Boston**    | `CrimeDataBoston`       | `dbfs:/mnt/training/crime-data-2016/Crime-Data-Boston-2016.parquet`

In [11]:
crimeDataNewYorkDF = spark.read.parquet("/mnt/training/crime-data-2016/Crime-Data-New-York-2016.parquet")

In [12]:
crimeDataBostonDF = spark.read.parquet("/mnt/training/crime-data-2016/Crime-Data-Boston-2016.parquet")

With the two DataFrames created, it is now possible to review the first couple records of each file.

Notice in the example below:
* The `crimeDataNewYorkDF` and `crimeDataBostonDF` DataFrames use different names for the columns.
* The data itself is formatted differently and different names are used for similar concepts.

This is common in a Data Lake. Often files are added to a Data Lake by different groups at different times. The advantage of this strategy is that anyone can contribute information to the Data Lake and that Data Lakes scale to store arbitrarily large and diverse data. The tradeoff for this ease in storing data is that it doesn’t have the rigid structure of a traditional relational data model, so the person querying the Data Lake will need to normalize data before extracting useful insights.

The alternative to a Data Lake is a data warehouse.  In a data warehouse, a committee often regulates the schema and ensures data is normalized before being made available.  This makes querying much easier but also makes gathering the data much more expensive and time-consuming.  Many companies choose to start with a Data Lake to accumulate data.  Then, as the need arises, they normalize data and produce higher quality tables for querying.  This reduces the up front costs while still making data easier to query over time.  The normalized tables can later be loaded into a formal data warehouse through nightly batch jobs.  In this way, Apache Spark is used to manage and query both Data Lakes and data warehouses.

In [14]:
display(crimeDataNewYorkDF)

complaintNumber,keyCode,offenseDescription,policeDeptCode,policeDeptDescription,lawCategoryCode,jurisdictionDesc,borough,precinct,locationOfOccurrenceDesc,premiseTypeDesc,latitude,longitude,fromDate,fromTime,toDate,toTime,reportDate
227505849,344,ASSAULT 3 & RELATED OFFENSES,101,ASSAULT 3,MISDEMEANOR,N.Y. POLICE DEPT,BRONX,52,INSIDE,CLOTHING/BOUTIQUE,40.86483612,-73.892447136,2016-12-31T00:00:00.000+0000,2016-12-31T16:40:00.000+0000,null,null,2016-12-31T00:00:00.000+0000
586068632,235,DANGEROUS DRUGS,511,"CONTROLLED SUBSTANCE, POSSESSI",MISDEMEANOR,PORT AUTHORITY,MANHATTAN,14,INSIDE,BUS TERMINAL,40.756266207,-73.990501248,2016-12-31T00:00:00.000+0000,2016-12-31T23:55:00.000+0000,2016-12-31T00:00:00.000+0000,2016-12-31T23:56:00.000+0000,2016-12-31T00:00:00.000+0000
155423129,105,ROBBERY,389,"ROBBERY,DWELLING",FELONY,N.Y. POLICE DEPT,BRONX,43,INSIDE,RESIDENCE - APT. HOUSE,40.828754623,-73.866593516,2016-12-31T00:00:00.000+0000,2016-12-31T23:40:00.000+0000,2016-12-31T00:00:00.000+0000,2016-12-31T23:50:00.000+0000,2016-12-31T00:00:00.000+0000
653964645,344,ASSAULT 3 & RELATED OFFENSES,101,ASSAULT 3,MISDEMEANOR,N.Y. POLICE DEPT,MANHATTAN,25,FRONT OF,STREET,40.809859893,-73.937644103,2016-12-31T00:00:00.000+0000,2016-12-31T23:30:00.000+0000,2016-12-31T00:00:00.000+0000,2016-12-31T23:31:00.000+0000,2016-12-31T00:00:00.000+0000
988275798,235,DANGEROUS DRUGS,567,"MARIJUANA, POSSESSION 4 & 5",MISDEMEANOR,N.Y. POLICE DEPT,MANHATTAN,7,OPPOSITE OF,STREET,40.719711494,-73.9894242,2016-12-31T00:00:00.000+0000,2016-12-31T23:25:00.000+0000,null,null,2016-12-31T00:00:00.000+0000
225104473,106,FELONY ASSAULT,109,"ASSAULT 2,1,UNCLASSIFIED",FELONY,N.Y. POLICE DEPT,QUEENS,102,null,STREET,40.694514975,-73.849134227,2016-12-31T00:00:00.000+0000,2016-12-31T23:24:00.000+0000,2016-12-31T00:00:00.000+0000,2016-12-31T23:30:00.000+0000,2016-12-31T00:00:00.000+0000
428909890,106,FELONY ASSAULT,109,"ASSAULT 2,1,UNCLASSIFIED",FELONY,N.Y. POLICE DEPT,BROOKLYN,70,INSIDE,RESIDENCE - APT. HOUSE,40.649370541,-73.960872294,2016-12-31T00:00:00.000+0000,2016-12-31T23:20:00.000+0000,2016-12-31T00:00:00.000+0000,2016-12-31T23:25:00.000+0000,2016-12-31T00:00:00.000+0000
313457048,344,ASSAULT 3 & RELATED OFFENSES,101,ASSAULT 3,MISDEMEANOR,N.Y. POLICE DEPT,BROOKLYN,79,INSIDE,RESIDENCE - APT. HOUSE,40.682000963,-73.948223153,2016-12-31T00:00:00.000+0000,2016-12-31T23:20:00.000+0000,2016-12-31T00:00:00.000+0000,2016-12-31T23:25:00.000+0000,2016-12-31T00:00:00.000+0000
816766111,126,MISCELLANEOUS PENAL LAW,198,CRIMINAL CONTEMPT 1,FELONY,N.Y. POLICE DEPT,MANHATTAN,13,null,STREET,40.73137039,-73.982563257,2016-12-31T00:00:00.000+0000,2016-12-31T23:19:00.000+0000,null,null,2016-12-31T00:00:00.000+0000
323812425,348,VEHICLE AND TRAFFIC LAWS,916,LEAVING SCENE-ACCIDENT-PERSONA,MISDEMEANOR,N.Y. POLICE DEPT,BROOKLYN,67,null,STREET,40.641135477,-73.945624473,2016-12-31T00:00:00.000+0000,2016-12-31T23:15:00.000+0000,2016-12-31T00:00:00.000+0000,2016-12-31T23:35:00.000+0000,2016-12-31T00:00:00.000+0000


In [15]:
display(crimeDataBostonDF)

INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,LATITUDE,LONGITUDE,LOCATION
I172071639,1402,Vandalism,VANDALISM,D4,905,null,2017-08-29T19:38:00.000+0000,2017,8,Tuesday,19,Part Two,HARRISON ARCHWAYS,42.33936768,-71.07035467,"(42.33936768, -71.07035467)"
I172071637,3114,Investigate Property,INVESTIGATE PROPERTY,E13,574,null,2017-08-29T16:00:00.000+0000,2017,8,Tuesday,16,Part Three,WASHINGTON ST,42.30971857,-71.10429432,"(42.30971857, -71.10429432)"
I172071635,3201,Property Lost,PROPERTY - LOST,A1,102,null,2017-08-29T21:00:00.000+0000,2017,8,Tuesday,21,Part Three,TREMONT ST,42.35637531,-71.06213513,"(42.35637531, -71.06213513)"
I172071633,3115,Investigate Person,INVESTIGATE PERSON,C11,240,null,2017-08-26T18:00:00.000+0000,2017,8,Saturday,18,Part Three,COLUMBIA RD,42.31959298,-71.062607,"(42.31959298, -71.06260700)"
I172071632,562,Other Burglary,BURGLARY - OTHER - NO FORCE,C6,232,null,2017-08-29T21:04:00.000+0000,2017,8,Tuesday,21,Part One,FARRAGUT RD,42.33370773,-71.02500441,"(42.33370773, -71.02500441)"
I172071631,619,Larceny,LARCENY ALL OTHERS,E5,727,null,2017-08-29T14:30:00.000+0000,2017,8,Tuesday,14,Part One,VFW PKWY,42.2802082,-71.17087959,"(42.28020820, -71.17087959)"
I172071630,613,Larceny,LARCENY SHOPLIFTING,E13,304,null,2017-08-29T21:16:00.000+0000,2017,8,Tuesday,21,Part One,COLUMBUS AVE,42.31771534,-71.09823049,"(42.31771534, -71.09823049)"
I172071629,3115,Investigate Person,INVESTIGATE PERSON,B3,465,null,2017-08-29T21:36:50.000+0000,2017,8,Tuesday,21,Part Three,BLUE HILL AVE,42.28482577,-71.09137369,"(42.28482577, -71.09137369)"
I172071624,1842,Drug Violation,"DRUGS - POSS CLASS A - HEROIN, ETC.",null,,null,2017-08-29T17:00:00.000+0000,2017,8,Tuesday,17,Part Two,null,42.31523705,-71.09777089,"(42.31523705, -71.09777089)"
I172071624,1849,Drug Violation,"DRUGS - POSS CLASS B - COCAINE, ETC.",null,,null,2017-08-29T17:00:00.000+0000,2017,8,Tuesday,17,Part Two,null,42.31523705,-71.09777089,"(42.31523705, -71.09777089)"


## Same Type of Data, Different Structure

In this section, we examine crime data to determine how to extract homicide statistics.

Because the data sets are pooled together in a Data Lake, each city may use different field names and values to indicate homicides, dates, etc.

For example:
* Some cities use the value "HOMICIDE", "CRIMINAL HOMICIDE" or "MURDER".
* In the New York data, the column is named `offenseDescription` while in the Boston data, the column is named `OFFENSE_CODE_GROUP`.
* In the New York data, the date of the event is in the `reportDate`, while in the Boston data, there is a single column named `MONTH`.

<iframe  
src="//fast.wistia.net/embed/iframe/ah2qrd2nh6?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/ah2qrd2nh6?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

To get started, create a temporary view containing only the homicide-related rows.

At the same time, normalize the data structure of each table so all the columns (and their values) line up with each other.

In the case of New York and Boston, here are the unique characteristics of each data set:

| | Offense-Column        | Offense-Value          | Reported-Column  | Reported-Data Type |
|-|-----------------------|------------------------|-----------------------------------|
| New York | `offenseDescription`  | starts with "murder" or "homicide" | `reportDate`     | `timestamp`    |
| Boston | `OFFENSE_CODE_GROUP`  | "Homicide"             | `MONTH`          | `integer`      |

For the upcoming aggregation, you need to alter the New York data set to include a `month` column which can be computed from the `reportDate` column using the `month()` function. Boston already has this column.

In this example, we use several functions in the `pyspark.sql.functions` library, and need to import:

* `month()` to extract the month from `reportDate` timestamp data type.
* `lower()` to convert text to lowercase.
* `contains(mySubstr)` to indicate a string contains substring `mySubstr`.

Also, note we use  `|`  to indicate a logical `or` of two conditions in the `filter` method.

In [19]:
from pyspark.sql.functions import lower, upper, month, col

homicidesNewYorkDF = (crimeDataNewYorkDF 
  .select(month(col("reportDate")).alias("month"), col("offenseDescription").alias("offense")) 
  .filter(lower(col("offenseDescription")).contains("murder") | lower(col("offenseDescription")).contains("homicide"))
)

display(homicidesNewYorkDF)

month,offense
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER


Notice how the same kind of information is presented differently in the Boston data:

`offense` is called `OFFENSE_CODE_GROUP` and there is only one category `homicide`.

In [21]:
homicidesBostonDF = (crimeDataBostonDF 
  .select("month", col("OFFENSE_CODE_GROUP").alias("offense")) 
  .filter(lower(col("OFFENSE_CODE_GROUP")).contains("homicide"))
)

display(homicidesBostonDF)

month,offense
8,Homicide
8,Homicide
8,Homicide
8,Homicide
8,Homicide
7,Homicide
7,Homicide
7,Homicide
7,Homicide
7,Homicide


See below the structure of the two tables is now identical.

In [23]:
display(homicidesNewYorkDF.limit(5))

month,offense
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER


In [24]:
display(homicidesBostonDF.limit(5))

month,offense
8,Homicide
8,Homicide
8,Homicide
8,Homicide
8,Homicide


## Analyzing the Data

-sandbox
Now that you normalized the homicide data for each city, combine the two by taking their union.

When done, aggregate that data to compute the number of homicides per month.

Start by creating a new DataFrame called `homicidesBostonAndNewYorkDF` that consists of the `union` of `homicidesNewYorkDF` with `homicidesBostonDF`.

<iframe  
src="//fast.wistia.net/embed/iframe/d76wihpsh6?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/d76wihpsh6?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

In [28]:
homicidesBostonAndNewYorkDF = homicidesNewYorkDF.union(homicidesBostonDF)

See all the data in one table below:

In [30]:
display(homicidesBostonAndNewYorkDF.orderBy("month"))

month,offense
1,MURDER & NON-NEGL. MANSLAUGHTER
1,MURDER & NON-NEGL. MANSLAUGHTER
1,MURDER & NON-NEGL. MANSLAUGHTER
1,MURDER & NON-NEGL. MANSLAUGHTER
1,MURDER & NON-NEGL. MANSLAUGHTER
1,MURDER & NON-NEGL. MANSLAUGHTER
1,MURDER & NON-NEGL. MANSLAUGHTER
1,MURDER & NON-NEGL. MANSLAUGHTER
1,MURDER & NON-NEGL. MANSLAUGHTER
1,MURDER & NON-NEGL. MANSLAUGHTER


And finally, perform a simple aggregation to see the number of homicides per month:

In [32]:
display(homicidesBostonAndNewYorkDF.select("month").orderBy("month").groupBy("month").count())

month,count
1,29
2,21
3,29
4,36
5,38
6,42
7,45
8,50
9,43
10,29


## Exercise 1

Merge the crime data for Chicago with the data for New York and Boston, and then update our final aggregation of counts-by-month.

### Step 1

Create the initial DataFrame of the Chicago data.
0. The source file is `dbfs:/mnt/training/crime-data-2016/Crime-Data-Chicago-2016.parquet`.
0. Name the view `crimeDataChicagoDF`.
0. View the data by invoking the `show()` method.

In [35]:
# TODO

crimeDataChicagoDF = spark.read.parquet('dbfs:/mnt/training/crime-data-2016/Crime-Data-Chicago-2016.parquet')
display(crimeDataChicagoDF)

id,caseNumber,block,iucr,primaryType,description,locationDescription,arrest,domestic,beat,district,ward,communityArea,fbiCode,xCoordinate,yCoordinate,year,updatedOn,latitude,longitude,location,date
10561135,HZ307126,009XX S HOMAN AVE,0560,ASSAULT,SIMPLE,SIDEWALK,false,false,1134,11,24,29,08A,1153851,1895648,2016,07/04/2016 03:55:03 PM,41.869491903,-87.710642788,"(41.869491903, -87.710642788)",2016-06-15T05:34:00.000+0000
10560338,HZ307136,040XX W GRAND AVE,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,true,false,2534,25,30,23,06,1148992,1909222,2016,06/21/2016 03:56:51 PM,41.90683577,-87.728130174,"(41.90683577, -87.728130174)",2016-06-15T05:34:00.000+0000
10560230,HZ307139,001XX N WABASH AVE,0810,THEFT,OVER $500,SIDEWALK,false,false,111,1,42,32,06,1176797,1901090,2016,06/21/2016 03:56:51 PM,41.883937418,-87.626237711,"(41.883937418, -87.626237711)",2016-06-15T05:35:00.000+0000
10560675,HZ307715,054XX S DEARBORN ST,0620,BURGLARY,UNLAWFUL ENTRY,APARTMENT,false,false,225,2,3,40,05,1176903,1869006,2016,06/21/2016 03:56:51 PM,41.795894095,-87.626817596,"(41.795894095, -87.626817596)",2016-06-15T05:37:00.000+0000
10560130,HZ307145,019XX N FRANCISCO AVE,0470,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,false,true,1421,14,35,22,24,1156824,1912657,2016,06/27/2016 03:50:03 PM,41.916106457,-87.69926659,"(41.916106457, -87.69926659)",2016-06-15T05:40:00.000+0000
10560258,HZ307129,065XX S DR MARTIN LUTHER KING JR DR,2820,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,false,true,312,3,20,42,26,1180047,1861654,2016,06/21/2016 03:56:51 PM,41.775648021,-87.615513559,"(41.775648021, -87.615513559)",2016-06-15T05:40:00.000+0000
10560272,HZ307112,012XX S AVERS AVE,143A,WEAPONS VIOLATION,UNLAWFUL POSS OF HANDGUN,SIDEWALK,true,false,1011,10,24,29,15,1150905,1894214,2016,07/09/2016 03:47:38 PM,41.865614975,-87.721495958,"(41.865614975, -87.721495958)",2016-06-15T05:45:00.000+0000
10560220,HZ307187,003XX N CENTRAL PARK AVE,0860,THEFT,RETAIL THEFT,SMALL RETAIL STORE,false,false,1123,11,28,27,06,1152330,1901857,2016,06/21/2016 03:56:51 PM,41.886560206,-87.716062887,"(41.886560206, -87.716062887)",2016-06-15T05:45:00.000+0000
10560113,HZ307128,0000X E ONTARIO ST,0860,THEFT,RETAIL THEFT,RESTAURANT,false,false,1834,18,42,8,06,1176729,1904501,2016,06/21/2016 03:56:51 PM,41.893298918,-87.626384218,"(41.893298918, -87.626384218)",2016-06-15T05:47:00.000+0000
10560275,HZ307252,031XX W 38TH PL,141A,WEAPONS VIOLATION,UNLAWFUL USE HANDGUN,ALLEY,false,false,911,9,12,58,15,1156127,1878918,2016,06/21/2016 03:56:51 PM,41.823537245,-87.702737932,"(41.823537245, -87.702737932)",2016-06-15T05:49:00.000+0000


In [36]:
# TEST - Run this cell to test your solution.

total = crimeDataChicagoDF.count()

dbTest("DF-L6-crimeDataChicago-count", 267872, total)

print("Tests passed!")

Tests passed!

-sandbox
### Step 2

Create a new view that normalizes the data structure.
0. Name the DataFrame `homicidesChicagoDF`.
0. The DataFrame should have at least two columns: `month` and `offense`.
0. Filter the data to include only homicides.
0. View the data by invoking the `show()` method.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** Use the `month()` function to extract the month-of-the-year.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** To find out which values for each offense constitutes a homicide, produce a distinct list of values from the`crimeDataChicagoDF` DataFrame.

In [38]:
# Get unique values
display(crimeDataChicagoDF.select("primaryType").distinct())

primaryType
OFFENSE INVOLVING CHILDREN
STALKING
PUBLIC PEACE VIOLATION
OBSCENITY
NON-CRIMINAL (SUBJECT SPECIFIED)
ARSON
GAMBLING
CRIMINAL TRESPASS
ASSAULT
NON - CRIMINAL


In [39]:
# TODO

homicidesChicagoDF = (crimeDataChicagoDF
                      .select(month(col("date")).alias("month"), col("primaryType").alias("offense"))
                      .filter(lower(col("offense")).contains("homicide"))
                     )

display(homicidesChicagoDF)

month,offense
6,HOMICIDE
6,HOMICIDE
6,HOMICIDE
6,HOMICIDE
6,HOMICIDE
6,HOMICIDE
6,HOMICIDE
6,HOMICIDE
6,HOMICIDE
6,HOMICIDE


In [40]:
# TEST - Run this cell to test your solution.
homicidesChicago = homicidesChicagoDF.select("month").groupBy("month").count().orderBy("month").collect()

dbTest("DF-L6-homicideChicago-len", 12, len(homicidesChicago))
dbTest("DF-L6-homicideChicago-0", 54, homicidesChicago[0][1])
dbTest("DF-L6-homicideChicago-6", 71, homicidesChicago[6][1])
dbTest("DF-L6-homicideChicago-11", 58, homicidesChicago[11][1])

print("Tests passed!")

Tests passed!

-sandbox
### Step 3

Create a new DataFrame that merges all three data sets (New York, Boston, Chicago):
0. Name the view `allHomicidesDF`.
0. Use the `union()` method introduced earlier to merge all three tables.
  * `homicidesNewYorkDF`
  * `homicidesBostonDF`
  * `homicidesChicagoDF`
0. View the data by invoking the `show()` method.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** To union three tables together, copy the previous example and apply a `union()` method again.

In [42]:
# TODO

allHomicidesDF = (homicidesNewYorkDF
                  .union(homicidesBostonDF)
                  .union(homicidesChicagoDF)
                 )

display(allHomicidesDF)

month,offense
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER
12,MURDER & NON-NEGL. MANSLAUGHTER


In [43]:
# TEST - Run this cell to test your solution.

allHomicides = allHomicidesDF.count()
dbTest("DF-L6-allHomicides-count", 1203, allHomicides)

print("Tests passed!")

Tests passed!

### Step 4

Create a new DataFrame that counts the number of homicides per month.
0. Name the DataFrame `homicidesByMonthDF`.
0. Rename the column `count(1)` to `homicides`.
0. Group the data by `month`.
0. Sort the data by `month`.
0. Count the number of records for each aggregate.
0. View the data by invoking the `show()` method.

In [45]:
# TODO
from pyspark.sql.functions import count

homicidesByMonthDF  = (allHomicidesDF
                      .groupby("month")
                      .count().alias("homicides")
                      .sort("month"))

display(homicidesByMonthDF)

month,count
1,83
2,68
3,72
4,76
5,105
6,120
7,116
8,144
9,109
10,109


In [46]:
# TEST - Run this cell to test your solution.
allHomicides = homicidesByMonthDF.collect()

dbTest("DF-L6-homicidesByMonth-len", 12, len(allHomicides))
dbTest("DF-L6-homicidesByMonth-0", 1, allHomicides[0][0])
dbTest("DF-L6-homicidesByMonth-11", 12, allHomicides[11][0])
dbTest("DF-L6-allHomicides-0", 83, allHomicides[0][1])
dbTest("DF-L6-allHomicides-1", 83, allHomicides[0][1])
dbTest("DF-L6-allHomicides-2", 68, allHomicides[1][1])
dbTest("DF-L6-allHomicides-3", 72, allHomicides[2][1])
dbTest("DF-L6-allHomicides-4", 76, allHomicides[3][1])
dbTest("DF-L6-allHomicides-5", 105, allHomicides[4][1])
dbTest("DF-L6-allHomicides-6", 120, allHomicides[5][1])
dbTest("DF-L6-allHomicides-7", 116, allHomicides[6][1])
dbTest("DF-L6-allHomicides-8", 144, allHomicides[7][1])
dbTest("DF-L6-allHomicides-9", 109, allHomicides[8][1])
dbTest("DF-L6-allHomicides-10", 109, allHomicides[9][1])
dbTest("DF-L6-allHomicides-11", 111, allHomicides[10][1])
dbTest("DF-L6-allHomicides-12", 90, allHomicides[11][1])

print("Tests passed!")

Tests passed!

## Summary

* Spark DataFrames allow you to easily manipulate data in a Data Lake.

## Review Questions
**Q:** What is a Data Lake?  
**A:** A Data Lake is a collection of data files gathered from various sources.  Spark loads each file as a table and then executes queries by joining and aggregating these files.

**Q:** What are advantages of Data Lakes over classic Data Warehouses?  
**A:** Data Lakes allow for large amounts of data to be aggregated from many sources with minimal preparatory steps.  Data Lakes also allow for very large files.  Powerful query engines such as Spark can read the diverse collection of files and execute complex queries efficiently.

**Q:** What are some advantages of Data Warehouses?  
**A:** Data warehouses are neatly curated to ensure data from all sources fit a common schema.  This makes them easy to query.

**Q:** What's the best way to combine the advantages of Data Lakes and Data Warehouses?  
**A:** Start with a Data Lake.  As you query, you will discover cases where the data needs to be cleaned, combined, and made more accessible.  Create periodic Spark jobs to read these raw sources and write new "golden" DataFrames that are cleaned and more easily queried.

## Next Steps
* Please complete this short [feedback survey](https://www.surveymonkey.com/r/WH9L2LX).  Your input is extremely important and will shape future development.
* Next, take what you learned about working data lakes and apply them in the [Capstone Project]($./07-Capstone-Project).

-sandbox
&copy; 2018 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>